# ASE vs RT-Ehrenfest Cross-Check

Compare MaxwellLink's Ehrenfest implementation with the ASE/Psi4 BOMD pathway.

> **Note**: Requires `psi4`, `ase`, and the HCN geometry file.

## 1. Configure the Models

In [ ]:
import numpy as np
import maxwelllink as mxl
from pathlib import Path

XYZ_PATH = Path("tests/data/hcn.xyz").resolve()

model_rt = mxl.RTEhrenfestModel(
    engine="psi4",
    molecule_xyz=str(XYZ_PATH),
    functional="b3lyp",
    basis="sto-3g",
    dt_rttddft_au=10.0,
    delta_kick_au=0.0,
    memory="2GB",
    verbose=False,
    remove_permanent_dipole=False,
    n_fock_per_nuc=1,
    n_elec_per_fock=1,
    homo_to_lumo=False,
    force_type="bo",
    partial_charges=[1.0, -1.0, 0.0],
)
model_rt.initialize(dt_new=10.0, molecule_id=0)

model_ase = mxl.ASEModel(
    atoms=str(XYZ_PATH),
    calculator="psi4",
    calc_kwargs="method=b3lyp, basis=sto-3g",
    charges="[1.0 -1.0 0.0]",
)
model_ase.initialize(dt_new=10.0, molecule_id=0)


## 2. Propagate and Compare

In [ ]:
n_steps = 10
field = np.array([1e-2, 1e-2, 0.0])
traj_rt = []
traj_ase = [model_ase._snapshot()["positions"] * 1.8897259886]

for _ in range(n_steps):
    model_rt.propagate(effective_efield_vec=field)
    traj_rt.append(model_rt.traj_R[-1])

    model_ase.propagate(effective_efield_vec=field)
    snapshot = model_ase._snapshot()
    traj_ase.append(snapshot["positions"] * 1.8897259886)

bond_rt = [np.linalg.norm(R[0] - R[1]) for R in traj_rt]
bond_ase = [np.linalg.norm(R[0] - R[1]) for R in traj_ase]

print("max bond deviation", np.max(np.abs(np.array(bond_rt) - np.array(bond_ase))))

import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.plot(bond_rt, label="RT-Ehrenfest (Psi4)")
plt.plot(bond_ase, label="ASE (Psi4)")
plt.xlabel("time step")
plt.ylabel("HC bond length (bohr)")
plt.legend()
plt.tight_layout()
plt.show()
